In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [40]:
admis = pd.read_csv('../Data Files/NSS_Hospital_Inpatient_Admissions.csv',
                    infer_datetime_format=True, parse_dates=[8,9],
                   dtype= {6:'str',12:'str',14:'str',15:'str'})
visits = pd.read_csv('../Data Files/NSS_PrimaryCare_Office_Visits.csv',
                     infer_datetime_format=True, parse_dates=[12],
                    dtype={8:'str',9:'str',10:'str',11:'str'})
mem = pd.read_csv('../Data Files/NSS_GreatCare_Members.csv',
                  infer_datetime_format=True, parse_dates=[5,61,62],
                 dtype= {6:'str',10:'str',11:'str',13:'str'})
npi = pd.read_csv('../output/npi.csv')
npi = npi.drop(columns=['Unnamed: 0'])
print(admis.shape)
print(visits.shape)
print(mem.shape)
print(npi.shape)
admisans = admis.copy()

C:\Users\pfbab\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3,5,17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(6213, 23)
(231373, 18)
(61519, 63)
(5415, 3)


In [41]:
visits = pd.merge(visits, npi, how='left', left_on='Service Provider NPI',right_on='npi' ).drop(columns=['entry','npi']).rename(columns={'enumeration':'Service Provider NPI Type'})
visits = pd.merge(visits, npi, how='left', left_on='Provider Group NPI',right_on='npi' ).drop(columns=['entry','npi']).rename(columns={'enumeration':'Provider Group NPI Type'})
admis = pd.merge(admis, npi, how='left', left_on='Attending Provider NPI',right_on='npi' ).drop(columns=['entry','npi']).rename(columns={'enumeration':'Attending Provider NPI Type'})
admis = pd.merge(admis, npi, how='left', left_on='Facility Provider NPI',right_on='npi' ).drop(columns=['entry','npi']).rename(columns={'enumeration':'Facility Provider NPI Type'})


In [47]:
visits.loc[visits['Provider Group NPI'].isna(),['Provider Group NPI Type']] = 'NPI-1'

NPI-2    226794
NPI-1      4526
NPI-0        53
Name: Provider Group NPI Type, dtype: int64

In [4]:
admisd = admis.sort_values(['Encounter Start Date','Encounter End Date']).drop_duplicates(subset=['Patient ID','Encounter End Date'], keep='first')
admisd = admisd.sort_values('Encounter End Date').drop_duplicates(subset=['Patient ID','Encounter Start Date'], keep='last')
admisd

,Encounter ID,Patient ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Facility Provider ID,Attending Provider ID,Attending Provider NPI,Facility Provider NPI,Encounter Start Date,Encounter End Date,...,CMS Place of Service Code,Principal Discharge Diagnosis ICD-9-CM Code,Principal Discharge Diagnosis ICD-9-CM Description,Principal Discharge Diagnosis ICD-10-CM Code,Principal Discharge Diagnosis ICD-10-CM Description,Primary Plan Allowed Amount,Primary Plan Paid Amount,Patient Paid Amount,Attending Provider NPI Type,Facility Provider NPI Type
77,HIA|DS1|10734|DS1|NPI|1104982917|1,DS1|10734,DS1|1,DS1|5047W,DS1|NPI|1104982917,DS1|NPI|1629382395,1629382395,1104982917,2009-11-11,2009-11-25,...,21,427.31,Atrial fibrillation,NaN,NaN,0,58730.78,0.0,NPI-1,NPI-2
1899,HIA|DS1|26295|DS1|NPI|1689608150|1,DS1|26295,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1346313590,1346313590,1689608150,2009-11-12,2009-11-30,...,NaN,779.89,Other specified conditions originating in the ...,NaN,NaN,0,32156.07,0.0,NPI-1,NPI-2
4982,HIA|DS1|53976|DS1|NPI|1689772592|1,DS1|53976,DS1|1,DS1|5047W,DS1|NPI|1689772592,DS1|NPI|1144488727,1144488727,1689772592,2009-11-29,2009-12-01,...,21,285.9,"Anemia, unspecified",NaN,NaN,0,9404.00,250.0,NPI-1,NPI-2
1304,HIA|DS1|21571|DS1|NPI|1689608150|1,DS1|21571,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1376637637,1376637637,1689608150,2009-11-27,2009-12-02,...,21,493.22,Chronic obstructive asthma with (acute) exacer...,NaN,NaN,0,23949.27,250.0,NPI-1,NPI-2
3429,HIA|DS1|40214|DS1|NPI|1710918545|1,DS1|40214,DS1|1,DS1|5047W,DS1|NPI|1710918545,DS1|NPI|1255314399,1255314399,1710918545,2009-12-03,2009-12-05,...,21,491.21,Obstructive chronic bronchitis with (acute) ex...,NaN,NaN,0,19378.00,250.0,NPI-1,NPI-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2428,HIA|DS1|31201|DS1|NPI|1184722779|1,DS1|31201,DS1|1,DS1|5047W,DS1|NPI|1184722779,DS1|NPI|1053334292,1053334292,1184722779,2019-02-15,2019-02-18,...,21,NaN,NaN,Z38.01,"Single liveborn infant, delivered by cesarean",0,2596.78,250.0,NPI-1,NPI-2
5905,HIA|DS1|7017|DS1|NPI|1184722779|1,DS1|7017,DS1|1,DS1|5047X,DS1|NPI|1184722779,DS1|NPI|1730127069,1730127069,1184722779,2019-02-14,2019-02-18,...,21,NaN,NaN,O43.123,"Velamentous insertion of umbilical cord, third...",0,20933.00,250.0,NPI-1,NPI-2
1335,HIA|DS1|2184|DS1|NPI|1902803315|2,DS1|2184,DS1|1,DS1|5047X,DS1|NPI|1902803315,DS1|NPI|1437358884,1437358884,1902803315,2019-02-24,2019-02-26,...,21,NaN,NaN,Z43.2,Encounter for attention to ileostomy,0,18750.00,250.0,NPI-1,NPI-2
4643,HIA|DS1|51247|DS1|NPI|1184722779|2,DS1|51247,DS1|1,DS1|5047W,DS1|NPI|1184722779,DS1|NPI|1851529879,1851529879,1184722779,2019-04-09,2019-04-13,...,21,NaN,NaN,T82.855A,"Stenosis of coronary artery stent, initial enc...",0,45083.79,202.0,NPI-1,NPI-2


In [5]:
visits.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 231373 entries, 0 to 231372
Data columns (total 20 columns):
 #   Column                                         Non-Null Count   Dtype         
---  ------                                         --------------   -----         
 0   Encounter ID                                   231373 non-null  object        
 1   Patient ID                                     231373 non-null  object        
 2   Service Provider ID                            230599 non-null  object        
 3   Attending Provider ID                          39644 non-null   object        
 4   Provider Group Provider ID                     227644 non-null  object        
 5   Facility Provider ID                           39756 non-null   object        
 6   Encounter Primary Payor ID                     231373 non-null  object        
 7   Encounter Primary Plan ID                      231373 non-null  object        
 8   Attending Provider NPI                      

In [6]:
visitscount = pd.DataFrame(visits['Patient ID'].value_counts()).reset_index().rename(columns={'index':'Patient ID','Patient ID':'count'})
visitscount

,Patient ID,count
0,DS1|44027,94
1,DS1|13892,81
2,DS1|51224,59
3,DS1|32475,57
4,DS1|29329,56
...,...,...
50729,DS1|34776,1
50730,DS1|59883,1
50731,DS1|15980,1
50732,DS1|25089,1


In [7]:
admiscount =  pd.DataFrame(admis['Patient ID'].value_counts()).reset_index().rename(columns={'index':'Patient ID','Patient ID':'count'})
admiscount[admiscount['count']>1]['Patient ID']

0       DS1|3812
1      DS1|26751
2      DS1|53241
3      DS1|10451
4      DS1|24041
         ...    
865    DS1|18134
866    DS1|53010
867    DS1|52876
868    DS1|23058
869    DS1|57492
Name: Patient ID, Length: 870, dtype: object

In [8]:
visits[visits['Patient ID'].isin(admiscount[admiscount['count']>5]['Patient ID'])]['Patient ID'].value_counts()

DS1|57685    39
DS1|3812     32
DS1|47885    32
DS1|11089    27
DS1|33966    25
DS1|60619    24
DS1|37477    22
DS1|24041    20
DS1|53241    19
DS1|12651    17
DS1|52751    15
DS1|5859     15
DS1|6506     14
DS1|10451    14
DS1|5182     12
DS1|51692    12
DS1|21571    11
DS1|11785    11
DS1|23411    11
DS1|3663     10
DS1|24805    10
DS1|32814    10
DS1|14274     9
DS1|27596     9
DS1|22069     8
DS1|15637     7
DS1|42969     7
DS1|55609     7
DS1|54840     6
DS1|48347     6
DS1|26751     5
DS1|53976     5
DS1|13593     5
DS1|36886     5
DS1|36688     4
DS1|48279     4
DS1|12422     4
DS1|53921     4
DS1|12364     3
DS1|21926     3
DS1|56584     2
DS1|20461     1
DS1|37954     1
DS1|40290     1
Name: Patient ID, dtype: int64

In [9]:
patient = admis[admis['Patient ID']=='DS1|24041'].sort_values('Encounter End Date').drop_duplicates(subset=['Patient ID','Encounter Start Date'], keep='last')
patient

,Encounter ID,Patient ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Facility Provider ID,Attending Provider ID,Attending Provider NPI,Facility Provider NPI,Encounter Start Date,Encounter End Date,...,CMS Place of Service Code,Principal Discharge Diagnosis ICD-9-CM Code,Principal Discharge Diagnosis ICD-9-CM Description,Principal Discharge Diagnosis ICD-10-CM Code,Principal Discharge Diagnosis ICD-10-CM Description,Primary Plan Allowed Amount,Primary Plan Paid Amount,Patient Paid Amount,Attending Provider NPI Type,Facility Provider NPI Type
1645,HIA|DS1|24041|DS1|NPI|1902803315|1,DS1|24041,DS1|1,DS1|5047W,DS1|NPI|1902803315,DS1|NPI|1013113679,1013113679,1902803315,2011-03-03,2011-03-09,...,21,571.2,Alcoholic cirrhosis of liver,NaN,NaN,0,40550.00,250.00,NPI-1,NPI-2
1636,HIA|DS1|24041|DS1|NPI|1427055839|1,DS1|24041,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1932307808,1932307808,1427055839,2011-04-22,2011-04-29,...,21,008.45,Intestinal infection due to Clostridium difficile,NaN,NaN,0,37900.00,250.00,NPI-1,NPI-2
1646,HIA|DS1|24041|DS1|NPI|1902844988|1,DS1|24041,DS1|1,DS1|5047W,DS1|NPI|1902844988,DS1|NPI|1619278108,1619278108,1902844988,2011-07-24,2011-07-29,...,23,584.9,"Acute kidney failure, unspecified",NaN,NaN,0,16759.32,247.99,NPI-1,NPI-2
1637,HIA|DS1|24041|DS1|NPI|1427055839|2,DS1|24041,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1679541775,1679541775,1427055839,2011-08-09,2011-08-12,...,21,584.9,"Acute kidney failure, unspecified",NaN,NaN,0,17000.79,17.01,NPI-1,NPI-2
1638,HIA|DS1|24041|DS1|NPI|1427055839|3,DS1|24041,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1942369152,1942369152,1427055839,2011-08-27,2011-09-01,...,21,572.2,Hepatic encephalopathy,NaN,NaN,0,27250.00,0.00,NPI-1,NPI-2
1639,HIA|DS1|24041|DS1|NPI|1427055839|4,DS1|24041,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1679541775,1679541775,1427055839,2011-11-21,2011-11-28,...,23,NaN,NaN,K70.31,Alcoholic cirrhosis of liver with ascites,0,46917.80,0.00,NPI-1,NPI-2
1640,HIA|DS1|24041|DS1|NPI|1427055839|5,DS1|24041,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1417061797,1417061797,1427055839,2012-01-02,2012-01-07,...,23,NaN,NaN,E87.1,Hypo-osmolality and hyponatremia,0,27917.80,0.00,NPI-1,NPI-2
1641,HIA|DS1|24041|DS1|NPI|1427055839|6,DS1|24041,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1053358127,1053358127,1427055839,2012-03-05,2012-03-06,...,NaN,NaN,NaN,N39.0,"Urinary tract infection, site not specified",0,5200.00,250.00,NPI-1,NPI-2
1642,HIA|DS1|24041|DS1|NPI|1467459776|1,DS1|24041,DS1|1,DS1|5047W,DS1|NPI|1467459776,DS1|NPI|1366717563,1366717563,1467459776,2012-04-03,2012-04-08,...,21,NaN,NaN,N39.0,"Urinary tract infection, site not specified",0,21095.00,250.00,NPI-1,NPI-2
1643,HIA|DS1|24041|DS1|NPI|1467459776|2,DS1|24041,DS1|1,DS1|5047W,DS1|NPI|1467459776,DS1|NPI|1659663979,1659663979,1467459776,2012-04-24,2012-05-01,...,21,NaN,NaN,N39.0,"Urinary tract infection, site not specified",0,29633.00,250.00,NPI-1,NPI-2


In [10]:
vpatient = visits[visits['Patient ID']=='DS1|24041'].sort_values('Encounter Date')
vpatient

,Encounter ID,Patient ID,Service Provider ID,Attending Provider ID,Provider Group Provider ID,Facility Provider ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Attending Provider NPI,Facility Provider NPI,Service Provider NPI,Provider Group NPI,Encounter Date,Is Qualifying Preventive Primary Care Service,Is Qualifying Primary Care Service Provider,Is Qualifying Primary Care Attending,Is Qualifying Primary Care Provider Group,Qualifying Preventive Primary Care HCPCS Code,Service Provider NPI Type,Provider Group NPI Type
90094,COV|PCSL|DS1|24041|P|804843|1043427248|1,DS1|24041,DS1|NPI|1750392247,NaN,DS1|NPI|1043427248,NaN,DS1|1,DS1|5047W,NaN,NaN,1750392247,1043427248,2011-04-01,0,1,0,0,NaN,NPI-1,NPI-2
90080,COV|PCSL|DS1|24041|P|1163045|1043427248|1,DS1|24041,DS1|NPI|1750392247,NaN,DS1|NPI|1043427248,NaN,DS1|1,DS1|5047W,NaN,NaN,1750392247,1043427248,2011-04-22,0,1,0,0,NaN,NPI-1,NPI-2
90086,COV|PCSL|DS1|24041|P|18283|1043427248|1,DS1|24041,DS1|NPI|1750392247,NaN,DS1|NPI|1043427248,NaN,DS1|1,DS1|5047W,NaN,NaN,1750392247,1043427248,2011-05-03,0,1,0,0,NaN,NPI-1,NPI-2
90095,COV|PCSL|DS1|24041|P|889934|1043427248|1,DS1|24041,DS1|NPI|1750392247,NaN,DS1|NPI|1043427248,NaN,DS1|1,DS1|5047W,NaN,NaN,1750392247,1043427248,2011-06-01,0,1,0,0,NaN,NPI-1,NPI-2
90092,COV|PCSL|DS1|24041|P|645576|1043427248|1,DS1|24041,DS1|NPI|1750392247,NaN,DS1|NPI|1043427248,NaN,DS1|1,DS1|5047W,NaN,NaN,1750392247,1043427248,2011-08-08,0,1,0,0,NaN,NPI-1,NPI-2
90088,COV|PCSL|DS1|24041|P|255825|1043427248|1,DS1|24041,DS1|NPI|1750392247,NaN,DS1|NPI|1043427248,NaN,DS1|1,DS1|5047W,NaN,NaN,1750392247,1043427248,2011-10-11,0,1,0,0,NaN,NPI-1,NPI-2
90093,COV|PCSL|DS1|24041|P|757527|1043427248|1,DS1|24041,DS1|NPI|1750392247,NaN,DS1|NPI|1043427248,NaN,DS1|1,DS1|5047W,NaN,NaN,1750392247,1043427248,2011-11-18,0,1,0,0,NaN,NPI-1,NPI-2
90082,COV|PCSL|DS1|24041|P|1230204|1043427248|1,DS1|24041,DS1|NPI|1588982904,NaN,DS1|NPI|1043427248,NaN,DS1|1,DS1|5047W,NaN,NaN,1588982904,1043427248,2012-03-15,0,1,0,0,NaN,NPI-1,NPI-2
90090,COV|PCSL|DS1|24041|P|35767|1043427248|1,DS1|24041,DS1|NPI|1588982904,NaN,DS1|NPI|1043427248,NaN,DS1|1,DS1|5047W,NaN,NaN,1588982904,1043427248,2012-03-22,0,1,0,0,NaN,NPI-1,NPI-2
90087,COV|PCSL|DS1|24041|P|251941|1043427248|1,DS1|24041,DS1|NPI|1588982904,NaN,DS1|NPI|1043427248,NaN,DS1|1,DS1|5047W,NaN,NaN,1588982904,1043427248,2012-03-29,0,1,0,0,NaN,NPI-1,NPI-2


In [11]:
vpatient
postvisit = 0
for rnum in range(0,len(patient)):

    print('rnum:' + str(rnum))
    trigger = vpatient[vpatient['Encounter Date']>=patient.iloc[rnum,9]]['Encounter Date']  <=  (patient.iloc[rnum,9] + pd.DateOffset(days=7))
    
    postvisit += int(trigger.sum() > 0)
    if trigger.sum() > 0:
        print('true')
        admisd.loc[admisd['Encounter ID']==patient.iloc[rnum,0],'pcp_visit'] = True
    else:
        print('false')
        admisd.loc[admisd['Encounter ID']==patient.iloc[rnum,0],'pcp_visit'] = False



rnum:0
false
rnum:1
true
rnum:2
false
rnum:3
false
rnum:4
false
rnum:5
false
rnum:6
false
rnum:7
false
rnum:8
true
rnum:9
false
rnum:10
false


In [12]:
postvisit

2

In [13]:
admisd[~admisd['pcp_visit'].isna()]

,Encounter ID,Patient ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Facility Provider ID,Attending Provider ID,Attending Provider NPI,Facility Provider NPI,Encounter Start Date,Encounter End Date,...,Principal Discharge Diagnosis ICD-9-CM Code,Principal Discharge Diagnosis ICD-9-CM Description,Principal Discharge Diagnosis ICD-10-CM Code,Principal Discharge Diagnosis ICD-10-CM Description,Primary Plan Allowed Amount,Primary Plan Paid Amount,Patient Paid Amount,Attending Provider NPI Type,Facility Provider NPI Type,pcp_visit
1645,HIA|DS1|24041|DS1|NPI|1902803315|1,DS1|24041,DS1|1,DS1|5047W,DS1|NPI|1902803315,DS1|NPI|1013113679,1013113679,1902803315,2011-03-03,2011-03-09,...,571.2,Alcoholic cirrhosis of liver,NaN,NaN,0,40550.00,250.00,NPI-1,NPI-2,False
1636,HIA|DS1|24041|DS1|NPI|1427055839|1,DS1|24041,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1932307808,1932307808,1427055839,2011-04-22,2011-04-29,...,008.45,Intestinal infection due to Clostridium difficile,NaN,NaN,0,37900.00,250.00,NPI-1,NPI-2,True
1646,HIA|DS1|24041|DS1|NPI|1902844988|1,DS1|24041,DS1|1,DS1|5047W,DS1|NPI|1902844988,DS1|NPI|1619278108,1619278108,1902844988,2011-07-24,2011-07-29,...,584.9,"Acute kidney failure, unspecified",NaN,NaN,0,16759.32,247.99,NPI-1,NPI-2,False
1637,HIA|DS1|24041|DS1|NPI|1427055839|2,DS1|24041,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1679541775,1679541775,1427055839,2011-08-09,2011-08-12,...,584.9,"Acute kidney failure, unspecified",NaN,NaN,0,17000.79,17.01,NPI-1,NPI-2,False
1638,HIA|DS1|24041|DS1|NPI|1427055839|3,DS1|24041,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1942369152,1942369152,1427055839,2011-08-27,2011-09-01,...,572.2,Hepatic encephalopathy,NaN,NaN,0,27250.00,0.00,NPI-1,NPI-2,False
1639,HIA|DS1|24041|DS1|NPI|1427055839|4,DS1|24041,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1679541775,1679541775,1427055839,2011-11-21,2011-11-28,...,NaN,NaN,K70.31,Alcoholic cirrhosis of liver with ascites,0,46917.80,0.00,NPI-1,NPI-2,False
1640,HIA|DS1|24041|DS1|NPI|1427055839|5,DS1|24041,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1417061797,1417061797,1427055839,2012-01-02,2012-01-07,...,NaN,NaN,E87.1,Hypo-osmolality and hyponatremia,0,27917.80,0.00,NPI-1,NPI-2,False
1641,HIA|DS1|24041|DS1|NPI|1427055839|6,DS1|24041,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1053358127,1053358127,1427055839,2012-03-05,2012-03-06,...,NaN,NaN,N39.0,"Urinary tract infection, site not specified",0,5200.00,250.00,NPI-1,NPI-2,False
1642,HIA|DS1|24041|DS1|NPI|1467459776|1,DS1|24041,DS1|1,DS1|5047W,DS1|NPI|1467459776,DS1|NPI|1366717563,1366717563,1467459776,2012-04-03,2012-04-08,...,NaN,NaN,N39.0,"Urinary tract infection, site not specified",0,21095.00,250.00,NPI-1,NPI-2,True
1643,HIA|DS1|24041|DS1|NPI|1467459776|2,DS1|24041,DS1|1,DS1|5047W,DS1|NPI|1467459776,DS1|NPI|1659663979,1659663979,1467459776,2012-04-24,2012-05-01,...,NaN,NaN,N39.0,"Urinary tract infection, site not specified",0,29633.00,250.00,NPI-1,NPI-2,False


In [14]:
patient.iloc[2,0]

'HIA|DS1|24041|DS1|NPI|1902844988|1'

In [15]:
dpatient = patient.set_index('Encounter End Date')['Encounter ID']
dpatient

Encounter End Date
2011-03-09    HIA|DS1|24041|DS1|NPI|1902803315|1
2011-04-29    HIA|DS1|24041|DS1|NPI|1427055839|1
2011-07-29    HIA|DS1|24041|DS1|NPI|1902844988|1
2011-08-12    HIA|DS1|24041|DS1|NPI|1427055839|2
2011-09-01    HIA|DS1|24041|DS1|NPI|1427055839|3
2011-11-28    HIA|DS1|24041|DS1|NPI|1427055839|4
2012-01-07    HIA|DS1|24041|DS1|NPI|1427055839|5
2012-03-06    HIA|DS1|24041|DS1|NPI|1427055839|6
2012-04-08    HIA|DS1|24041|DS1|NPI|1467459776|1
2012-05-01    HIA|DS1|24041|DS1|NPI|1467459776|2
2012-07-04    HIA|DS1|24041|DS1|NPI|1467459776|3
Name: Encounter ID, dtype: object

In [16]:
dvpatient = vpatient.set_index('Encounter Date')['Encounter ID']
dvpatient

Encounter Date
2011-04-01     COV|PCSL|DS1|24041|P|804843|1043427248|1
2011-04-22    COV|PCSL|DS1|24041|P|1163045|1043427248|1
2011-05-03      COV|PCSL|DS1|24041|P|18283|1043427248|1
2011-06-01     COV|PCSL|DS1|24041|P|889934|1043427248|1
2011-08-08     COV|PCSL|DS1|24041|P|645576|1043427248|1
2011-10-11     COV|PCSL|DS1|24041|P|255825|1043427248|1
2011-11-18     COV|PCSL|DS1|24041|P|757527|1043427248|1
2012-03-15    COV|PCSL|DS1|24041|P|1230204|1043427248|1
2012-03-22      COV|PCSL|DS1|24041|P|35767|1043427248|1
2012-03-29     COV|PCSL|DS1|24041|P|251941|1043427248|1
2012-04-12    COV|PCSL|DS1|24041|P|1305662|1043427248|1
2012-06-06    COV|PCSL|DS1|24041|P|1288602|1043427248|1
2012-08-02    COV|PCSL|DS1|24041|P|1019894|1043427248|1
2012-08-10     COV|PCSL|DS1|24041|P|972291|1043427248|1
2012-08-17    COV|PCSL|DS1|24041|P|1018549|1043427248|1
2012-09-03    COV|PCSL|DS1|24041|P|1218703|1043427248|1
2012-09-25       COV|PCSL|DS1|24041|P|1349|1043427248|1
2012-10-25    COV|PCSL|DS1|24041|

In [17]:
visittimeline = dpatient.append(dvpatient)

In [18]:
visittimeline.sort_index(inplace=True)
visittimeline

2011-03-09           HIA|DS1|24041|DS1|NPI|1902803315|1
2011-04-01     COV|PCSL|DS1|24041|P|804843|1043427248|1
2011-04-22    COV|PCSL|DS1|24041|P|1163045|1043427248|1
2011-04-29           HIA|DS1|24041|DS1|NPI|1427055839|1
2011-05-03      COV|PCSL|DS1|24041|P|18283|1043427248|1
2011-06-01     COV|PCSL|DS1|24041|P|889934|1043427248|1
2011-07-29           HIA|DS1|24041|DS1|NPI|1902844988|1
2011-08-08     COV|PCSL|DS1|24041|P|645576|1043427248|1
2011-08-12           HIA|DS1|24041|DS1|NPI|1427055839|2
2011-09-01           HIA|DS1|24041|DS1|NPI|1427055839|3
2011-10-11     COV|PCSL|DS1|24041|P|255825|1043427248|1
2011-11-18     COV|PCSL|DS1|24041|P|757527|1043427248|1
2011-11-28           HIA|DS1|24041|DS1|NPI|1427055839|4
2012-01-07           HIA|DS1|24041|DS1|NPI|1427055839|5
2012-03-06           HIA|DS1|24041|DS1|NPI|1427055839|6
2012-03-15    COV|PCSL|DS1|24041|P|1230204|1043427248|1
2012-03-22      COV|PCSL|DS1|24041|P|35767|1043427248|1
2012-03-29     COV|PCSL|DS1|24041|P|251941|10434

In [19]:
for rnum in range(0,len(visittimeline)-1):
    if (visittimeline[rnum].startswith('HIA') & visittimeline[rnum+1].startswith('COV'))&(visittimeline.index[rnum+1] <= (visittimeline.index[rnum])+pd.DateOffset(days=7)):
        print('true')
        #admis.loc[admis['Encounter ID']==visittimeline[rnum],'pcp_visit'] = True
    elif visittimeline[rnum].startswith('HIA'):
        print('false')
        #admis.loc[admis['Encounter ID']==visittimeline[rnum],'pcp_visit'] = False

false
true
false
false
false
false
false
false
true
false
false


In [20]:
pcp_visit = 0
for patnum in admis['Patient ID'].drop_duplicates():
    #create dataframe for each patient number keeping only the final discharge per Start Date
    patient = admis[admis['Patient ID']==patnum]
    patient = patient.sort_values(['Encounter Start Date','Encounter End Date']).drop_duplicates(subset=['Patient ID','Encounter End Date'], keep='first')
    patient = patient.sort_values('Encounter End Date').drop_duplicates(subset=['Patient ID','Encounter Start Date'], keep='last')
    vpatient = visits[visits['Patient ID']==patnum].sort_values('Encounter Date')
    dpatient = patient.set_index('Encounter End Date')['Encounter ID']
    dvpatient = vpatient.set_index('Encounter Date')['Encounter ID']
    
    visittimeline = pd.concat([dpatient ,dvpatient])
    visittimeline.sort_index(inplace=True)
    
    for rnum in range(0,len(visittimeline)-1):
        if (visittimeline[rnum].startswith('HIA') & visittimeline[rnum+1].startswith('COV'))&(visittimeline.index[rnum+1] <= (visittimeline.index[rnum])+pd.DateOffset(days=7)):
            pcp_visit += 1
            admis.loc[admis['Encounter ID']==visittimeline[rnum],'pcp_visit'] = True
            if (visits[visits['Encounter ID']==visittimeline[rnum+1]]['Provider Group NPI Type'].values[0] == 'NPI-1') | (visits[visits['Encounter ID']==visittimeline[rnum+1]]['Provider Group NPI'].isna().values[0]):
                admis.loc[admis['Encounter ID']==visittimeline[rnum],'pcp_visit_type'] = 'Individual'
            else:
                admis.loc[admis['Encounter ID']==visittimeline[rnum],'pcp_visit_type'] = 'Group'
        elif visittimeline[rnum].startswith('HIA'):
            admis.loc[admis['Encounter ID']==visittimeline[rnum],'pcp_visit'] = False


In [21]:
admis.pcp_visit.value_counts()

False    3256
True     1846
Name: pcp_visit, dtype: int64

In [22]:
admis.pcp_visit_type.value_counts()

Group         1817
Individual      29
Name: pcp_visit_type, dtype: int64

In [23]:
visittimeline.keys()[12]

IndexError: index 12 is out of bounds for axis 0 with size 1

In [ ]:
vpatient[vpatient['Encounter Date'].ge(visittimeline.keys()[12])].iloc[0]['Provider Group NPI Type']
